plotting for Paper 3

In [1]:
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import init_notebook_mode 

In [8]:
# Define common variables
outpath = '../output/'
fig_path = outpath + 'figs/'
data_path = '../data/'
import datetime
from utils.signatures import update_cumul_df, load_flow_loc

In [2]:
# import data for analysis
from common_settings import  outpath, \
    obs_events, day_load_flow, mod_load_flow

## Visualize the DIN process of selected events. 

In [4]:
# import EMC and tranform into a matrix
emc_obs = pd.read_csv(f'{outpath}obs_storm_event_common4.csv', index_col='ID')
emc_mod = pd.read_csv(f'{outpath}DIN_mod_storm_event_common4.csv', index_col='ID')
# Read observed concentration data.
fn_conc = 'gbr_WhiSun.xlsx'
conc_obs = pd.read_excel(f'{data_path}obs/{fn_conc}', sheet_name='126001A', index_col='DateTime')
conc_obs['Oxidised nitrogen (mg/L)'].replace('<0.001', 0.0005)
conc_obs['Ammonium nitrogen (mg/L)'].replace('<0.002', 0.001)
conc_obs['DIN (mg/L)'] = conc_obs['Oxidised nitrogen (mg/L)'].replace('<0.001', 0.0005) + conc_obs['Ammonium nitrogen (mg/L)'].replace('<0.002', 0.001)
conc_obs.index = pd.to_datetime(conc_obs.index)
# Read modeled concentration data.
conc_mod = pd.read_csv(f'{data_path}mod/DIN_flow.csv', index_col='Date')
conc_mod.index = pd.to_datetime(conc_mod.index)

conc_obsest = pd.read_csv(f'{data_path}obs/low_interp_flow.csv', index_col='Date')
conc_obsest.index = pd.to_datetime(conc_obsest.index, yearfirst=True)

# Read hourly flow
hour_flow = pd.read_csv(f'{data_path}obs/126001A_hourly.csv', index_col='Time')

# Convert start and end time of events into datetime.
emc_obs['start'] = pd.to_datetime(emc_obs['start'])
emc_obs['end'] = pd.to_datetime(emc_obs['end'])
emc_mod['start'] = pd.to_datetime(emc_mod['start'])
emc_mod['end'] = pd.to_datetime(emc_mod['end'])

day_load_flow['Flow (ML)'] = day_load_flow['Flow (ML)'] / 86.4
mod_load_flow['Flow (ML)'] = mod_load_flow['Flow (ML)'] / 86.4

In [5]:
# read information of the selected events
Event_ID = [1, 2, 3] # 45,
event_dict = {}
conc_dict = {}
for i in Event_ID:
    event_dict[f'event_obs_{i}'] = day_load_flow[emc_obs.loc[i, 'start']:emc_obs.loc[i, 'end']].filter(items=['Loads (kg)', 'Flow (ML)'])
    event_dict[f'event_mod_{i}'] = mod_load_flow[emc_obs.loc[i, 'start']:emc_obs.loc[i, 'end']].filter(items=['Loads (kg)', 'Flow (ML)'])
    conc_dict[f'event_obs_{i}'] = conc_obs[emc_obs.loc[i, 'start']:(emc_obs.loc[i, 'end'] + datetime.timedelta(days=1))].filter(items=['DIN (mg/L)'])
    conc_dict[f'event_mod_{i}'] = conc_mod[emc_obs.loc[i, 'start']:(emc_obs.loc[i, 'end'] + datetime.timedelta(days=1))].filter(items=['Concentration (mg/L)'])
    conc_dict[f'event_obsest_{i}'] = conc_obsest[emc_obs.loc[i, 'start']:(emc_obs.loc[i, 'end'] + datetime.timedelta(days=1))].filter(items=['Concentration (mg/L)'])


In [8]:
# read information of the selected events
# Determine whehter to change the yaxis of conc to a log type.
logy = True
for e_id in range(1, 55, 3):
        Event_ID = [e_id, e_id+1, e_id+2] # 45,
        event_dict = {}
        conc_dict = {}
        for i in Event_ID:
                event_dict[f'event_obs_{i}'] = day_load_flow[emc_obs.loc[i, 'start']:emc_obs.loc[i, 'end']].filter(items=['Loads (kg)', 'Flow (ML)'])
                event_dict[f'event_mod_{i}'] = mod_load_flow[emc_mod.loc[i, 'start']:emc_mod.loc[i, 'end']].filter(items=['Loads (kg)', 'Flow (ML)'])
                conc_dict[f'event_obs_{i}'] = conc_obs[emc_obs.loc[i, 'start']:(emc_obs.loc[i, 'end'] + datetime.timedelta(days=1))].filter(items=['DIN (mg/L)'])
                conc_dict[f'event_mod_{i}'] = conc_mod[emc_mod.loc[i, 'start']:(emc_mod.loc[i, 'end'] + datetime.timedelta(days=1))].filter(items=['Concentration (mg/L)'])
                conc_dict[f'event_obsest_{i}'] = conc_obsest[emc_obs.loc[i, 'start']:(emc_obs.loc[i, 'end'] + datetime.timedelta(days=1))].filter(items=['Concentration (mg/L)'])

        labs = ['Loads (kg)', 'Flow (ML)']
        fig = make_subplots(rows=3, cols=3, shared_yaxes=False, shared_xaxes=True, 
                                horizontal_spacing = 0.05, vertical_spacing = 0.01,
                                column_titles=[f'Event {i}'for i in Event_ID], row_heights=[1, 1, 1], column_widths=[1, 1, 1])
        show_legend = [True, False, False]
        for i in range(len(Event_ID)):
                x_obs = event_dict[f'event_obs_{Event_ID[i]}'].index
                x_mod = event_dict[f'event_mod_{Event_ID[i]}'].index
                fig.add_trace(go.Line(x = x_obs, y=event_dict[f'event_obs_{Event_ID[i]}'].loc[:, labs[1]], \
                                name=f'Obs {labs[1]}', line={'color': 'blue'}, showlegend = show_legend[i], mode='lines+markers', opacity=0.7), row=1, col= i+1)

                fig.add_trace(go.Line(x = x_mod, y=event_dict[f'event_mod_{Event_ID[i]}'].loc[:, labs[1]], \
                                name=f'Mod {labs[1]}', line={'dash' : 'dot', 'color': 'orange'}, showlegend = show_legend[i], mode='lines+markers', opacity=0.7), row=1, col= i+1)

                fig.add_trace(go.Line(x = x_obs, y=event_dict[f'event_obs_{Event_ID[i]}'].loc[:, labs[0]], \
                                name=f'Obs {labs[0]}', line={'color': 'blue'}, showlegend = show_legend[i], mode='lines+markers', opacity=0.7), \
                                row=2, col= i+1)
                fig.add_trace(go.Line(x = x_mod, y=event_dict[f'event_mod_{Event_ID[i]}'].loc[:, labs[0]], \
                                name=f'Mod {labs[0]}', line={'dash': 'dot', 'color': 'orange'}, showlegend = show_legend[i], mode='lines+markers', opacity=0.7), \
                                row=2, col= i+1)
                
                fig.add_trace(go.Line(x = conc_dict[f'event_obs_{Event_ID[i]}'].index, y=conc_dict[f'event_obs_{Event_ID[i]}'].loc[:, 'DIN (mg/L)'], \
                                name='Obs Conc  (mg/L)', line={'color': 'purple'}, showlegend = show_legend[i], mode='markers', marker={'symbol':'cross'}, opacity=0.5), \
                                row=3, col= i+1)
                fig.add_trace(go.Line(x = x_mod, y=conc_dict[f'event_mod_{Event_ID[i]}'].loc[:, 'Concentration (mg/L)'], \
                                name='Mod Conc (mg/L)', line={'color': 'orange'}, showlegend = show_legend[i], marker={'symbol':'cross'}, mode='markers', opacity=0.7), \
                                row=3, col= i+1)
                fig.add_trace(go.Line(x = x_obs, y=conc_dict[f'event_obsest_{Event_ID[i]}'].loc[:, 'Concentration (mg/L)'], \
                                name='Obs Estimate (mg/L)', line={'color': 'blue'}, showlegend = show_legend[i], mode='markers', marker={'symbol':'cross'}, opacity=0.7), \
                                row=3, col= i+1)
                fig["layout"]["yaxis"].update({"title": "Flow (Cumsec)"})
                fig["layout"]["yaxis4"].update({"title": "Load (Kg)"})
                fig["layout"]["yaxis7"].update({"title": "Conc (gm/L)"})
                if logy:
                        fig["layout"]["yaxis7"].update({'type':'log'})
                        fig["layout"]["yaxis8"].update({'type':'log'})
                        fig["layout"]["yaxis9"].update({'type':'log'})
                # layout = fig.update_layout(yaxis=dict(title=['Flow (ML)', 'Load (kg)', 'Conc (mg/L)']))
                fig.update_layout(autosize=False, width=1000, height=800,)
        fig.write_image(fig_path + f'event/event-scale-processes{Event_ID[0]}-{Event_ID[1]}-{Event_ID[2]}-logy.png', scale=2);

c:\Users\wq\.conda\envs\oed\lib\site-packages\plotly\graph_objs\_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




## Create observed concentration vs. flow.

In [8]:
# read information of the selected events
# Determine whehter to change the yaxis of conc to a log type.
logy = False
for e_id in range(1, 4, 3):
        # Event_ID = [e_id, e_id+1, e_id+2] # 45,
        Event_ID = [28, 32, 42, 3]
        event_dict = {}
        conc_dict = {}
        for i in Event_ID:
                event_dict[f'event_obs_{i}'] = day_load_flow[emc_obs.loc[i, 'start']:emc_obs.loc[i, 'end']].filter(items=['Loads (kg)', 'Flow (ML)'])
                event_dict[f'event_mod_{i}'] = mod_load_flow[emc_mod.loc[i, 'start']:emc_mod.loc[i, 'end']].filter(items=['Loads (kg)', 'Flow (ML)'])
                conc_dict[f'event_obs_{i}'] = conc_obs[emc_obs.loc[i, 'start']:(emc_obs.loc[i, 'end'] + datetime.timedelta(days=1))].filter(items=['DIN (mg/L)'])
                conc_dict[f'event_mod_{i}'] = conc_mod[emc_mod.loc[i, 'start']:(emc_mod.loc[i, 'end'] + datetime.timedelta(days=1))].filter(items=['Concentration (mg/L)'])
                conc_dict[f'event_obsest_{i}'] = conc_obsest[emc_obs.loc[i, 'start']:(emc_obs.loc[i, 'end'] + datetime.timedelta(days=1))].filter(items=['Concentration (mg/L)'])

        labs = ['Loads (kg)', 'Flow (ML)']
        fig = make_subplots(rows=2, cols=4, shared_yaxes=False, shared_xaxes=True, 
                                horizontal_spacing = 0.05, vertical_spacing = 0.01,
                                column_titles=[f'Event {i}'for i in Event_ID], row_heights=[1, 1], column_widths=[1, 1, 1, 1])
        show_legend = [True, False, False, False]
        for i in range(len(Event_ID)):
                x_obs = event_dict[f'event_obs_{Event_ID[i]}'].index
                x_mod = event_dict[f'event_mod_{Event_ID[i]}'].index
                fig.add_trace(go.Line(x = x_obs, y=event_dict[f'event_obs_{Event_ID[i]}'].loc[:, labs[1]], \
                                name=f'Obs flow (cumec)', line={'color': 'blue'}, showlegend = show_legend[i], mode='lines+markers', opacity=0.7), row=1, col= i+1)

                fig.add_trace(go.Line(x = x_mod, y=event_dict[f'event_mod_{Event_ID[i]}'].loc[:, labs[1]], \
                                name=f'Mod flow (cumec)', line={'dash' : 'dot', 'color': 'orange'}, showlegend = show_legend[i], mode='lines+markers', opacity=0.7), row=1, col= i+1)
                
                fig.add_trace(go.Line(x = conc_dict[f'event_obs_{Event_ID[i]}'].index, y=conc_dict[f'event_obs_{Event_ID[i]}'].loc[:, 'DIN (mg/L)'], \
                                name='Obs conc (mg/L)', line={'color': 'purple'}, showlegend = show_legend[i], mode='markers', marker={'symbol':'cross'}, opacity=0.5), \
                                row=2, col= i+1)
                fig.add_trace(go.Line(x = x_mod, y=conc_dict[f'event_mod_{Event_ID[i]}'].loc[:, 'Concentration (mg/L)'], \
                                name='Mod conc (mg/L)', line={'color': 'orange'}, showlegend = show_legend[i], marker={'symbol':'cross'}, mode='markers', opacity=0.7), \
                                row=2, col= i+1)
                fig["layout"]["yaxis"].update({"title": "Flow (cumec)"})
                fig["layout"]["yaxis5"].update({"title": "Conc (gm/L)"})
                if logy:
                        fig["layout"]["yaxis4"].update({'type':'log'})
                        fig["layout"]["yaxis5"].update({'type':'log'})
                        fig["layout"]["yaxis6"].update({'type':'log'})
                # layout = fig.update_layout(yaxis=dict(title=['Flow (ML)', 'Load (kg)', 'Conc (mg/L)']))
                fig.update_layout(autosize=False, width=1200, height=600,)
        fig.write_image(fig_path + f'event-conc-flow{Event_ID[0]}-{Event_ID[1]}-{Event_ID[2]}.png', scale=2);

## Create the double-mass plot for each component model.

In [4]:
from plotly.subplots import make_subplots
color_list = px.colors.qualitative.D3
xlabel='Normalized cumulative discharge'
ylabel='Normalized cumulative mass'
# Read the original contribution file containing parameter names.
para_all = pd.read_csv(f'{outpath}contribution_each_param.csv').columns[2:]

In [5]:
# To loop over parameters
for p_name in para_all:
    print(p_name)
    # Read excel
    df_mod = pd.read_excel(f'{outpath}{p_name}_cumulative_ratio_day.xlsx', None)
    # reshape the dataframe for plotting
    # Formatting model simulations
    df_mod_plot = pd.DataFrame(columns = ['Date', 'cumul_flow_ratio', 'cumul_load_ratio'])
    for k, v in df_mod.items():
        df_mod_plot = pd.concat([df_mod_plot, v.loc[:, ['Date', *v.columns[-2:]]]])
    df_mod_plot.set_index('Date', inplace=True)
    df_mod_plot.index= pd.to_datetime(df_mod_plot.index)

    fig = make_subplots(
        rows=1, cols=1,
        horizontal_spacing = 0.01,
        specs=[[{"type": "scatter"}]]
    )
        
    for ii in range(2009, 2018):
        # Plot simulations
        df_mod_temp = df_mod_plot.loc[pd.to_datetime(f'{ii}-07-01'):pd.to_datetime(f'{ii+1}-07-01'), :].values
        fig.add_trace(go.Scatter(x = df_mod_temp[:, 0], y=df_mod_temp[:, 1], mode = 'lines', name=f'{ii}',
                                line = dict(width=2, dash='dot', color = color_list[ii-2009])), 
                    row=1, col=1)

    fig.update_layout(
        yaxis = dict(
            title_text = ylabel
        ),
        xaxis = dict(
            title_text = xlabel
        ),
        title=p_name
    )

    fig.update_layout(height=500, width=800,
                    legend = dict(yanchor="bottom", y = 0.01, xanchor = "right", x = 0.99, 
                    font = dict(size=10), orientation='v', bgcolor="white",
                    bordercolor="Black",
                    borderwidth=1), legend_title_text='Year', font_family = "Arial", font_color = 'black')

    fig.write_image(f'{fig_path}{p_name}_cumulative_lines_.png', scale=2)

DeliveryRatioSurface
DeliveryRatioSeepage
Forestry-dissConst_DWC
Forestry-dissConst_EMC
Grazing Forested-dissConst_DWC
Grazing Forested-dissConst_EMC
Grazing Open-dissConst_DWC
Grazing Open-dissConst_EMC
Conservation-dissConst_DWC
Conservation-dissConst_EMC
Urban-dissConst_DWC
Urban-dissConst_EMC
Horticulture-dissConst_DWC
Horticulture-dissConst_EMC
DWC


## Create plot showing seasonal loads

In [13]:
data_mon = pd.read_csv(f'{outpath}month_loads_sources.csv')
df_seasonal = {'mod':{}, 'obs':{}}

In [4]:
# Calculate the seasonal din loads
for ii in range(0, data_mon.index[-1], 3):
    t_mon = data_mon.Month[ii]
    df_seasonal['mod'][t_mon] = data_mon.loc[ii:(ii+3), 'mod_all'].sum()
    df_seasonal['obs'][t_mon] = data_mon.loc[ii:(ii+3), 'obs'].sum()

# Convert the dict to dataframe
df_seasonal = pd.DataFrame.from_dict(df_seasonal)
df_seasonal.reset_index(inplace=True)
df_seasonal.head()


,index,mod,obs
0,2009/7,11.325321,260.2713
1,2009/10,13719.316663,27558.9787
2,2010/1,49570.462831,45491.0927
3,2010/4,1813.082779,7858.4604
4,2010/7,3618.200957,6427.4438


In [9]:
df_plot = df_seasonal.loc[1::4, :] # Select seasons Oct.-Dec.
# Add the number of samples.
# Read the file containing raw samples.
fn = 'gbr_WhiSun.xlsx'
dir_data = '../data/obs/'
conc = pd.read_excel(f'{dir_data}{fn}', sheet_name='126001A')
conc['DateTime'] = pd.to_datetime(conc['DateTime'])
wq_numbers = []
# Count the #samples.
for year in range(2009, 2018):
    t_start, t_end = pd.to_datetime(f'{year}-10-01'), pd.to_datetime(f'{year}-12-31')
    wq_samples = conc[conc['DateTime'] >= t_start][conc['DateTime'] < t_end + datetime.timedelta(days=1)];
    wq_numbers.append(wq_samples.shape[0])
df_plot.loc[:, 'sample_counts'] = wq_numbers

In [50]:
# Add sample counts in the figure.
fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df_plot['index'], y=df_plot['obs'], name='obs'), secondary_y=False)
fig.add_trace(go.Bar(x=df_plot['index'], y=df_plot['mod'], name='mod'), secondary_y=False)
fig.add_trace(go.Line(x=df_plot['index'], y=df_plot['sample_counts'], name='sample_counts'), secondary_y=True)
fig.update_layout(yaxis=dict(title_text='DIN load (KG)'), xaxis=dict(title_text='Time'))
fig.update_yaxes(title_text="Number of conc. samples", secondary_y=True)
fig.write_image(f'{fig_path}wet_seasonal_load_comparison.png', format='png', scale=2)